In [22]:
# evaluation mode when truth labels are provided in test data. 
!python build_index.py --data_dir /Users/a656526/Downloads/spellcheck/ \
                    --train_file NES_Export_Active_Intents_Only.csv \
                    --test_file TruthFileV2.csv \
                    --raw_or_clean clean --clean_filename corpus
                 


Reading pre-processed clean file as input ..

Building inverted index

Saving the inverted index as pickle
Process Done, success!


### 1) start with searched query

In [1]:
# import query_search
from query_search import *
#import pandas as pd
pd.set_option('display.max_colwidth', -1)
#search_docs(["how do i"])


search_docs(queries=["transfer","roth ira","tax form","i want to","how do i","when will","hsa","529"])

,query,suggested_docs,score
0,transfer,I want to set up a transfer,0.0297
1,transfer,I have a question about automatic transfers,0.0041
2,transfer,How do I transfer a custodial account to the minor?,0.0009
3,transfer,Does Fidelity accept payments from money transfer apps?,0.0006
4,transfer,What is Money Transfer Lockdown?,0.0005
5,roth ira,What are the rules for a Roth IRA?,0.0007
6,roth ira,What's the difference between a traditional and Roth IRA?,0.0006
7,roth ira,Do you offer Roth IRAs for kids?,0.0005
8,roth ira,What is a Roth IRA?,0.0005
9,roth ira,Does it cost a fee to open another Roth IRA account.,0.0000


In [4]:
import pandas as pd
import numpy as np
f=pd.read_csv("/Users/a656526/Downloads/spellcheck/NES_Export_Active_Intents_Only.csv")
t1=f.groupby(['Intent','DisplayExample']).size().reset_index(drop=False)
t1.columns=["Intent","Sentence","freq"]
t1['wgt']=round(t1['freq']/sum(t1.freq),4)
t1.drop(columns=["freq"],inplace=True)
t1.head()

,Intent,Sentence,wgt
0,ABANDONED_PROPERTY,I received an abandoned property notice,0.0018
1,ABC_MARKET_NEWS,What is the market news?,0.0024
2,ACAT_DEFINITION,What is ACAT?,0.0005
3,ACCESSIBILITY_ISSUE,I have an accessibility issue,0.0006
4,ACCT_BALANCE,Can I view my account balance?,0.0068


In [10]:

t4=pd.read_excel("/Users/a656526/Downloads/spellcheck/LRC-1.xlsx")
t5=pd.read_excel("/Users/a656526/Downloads/spellcheck/LRC-2.xlsx")
t6=pd.read_excel("/Users/a656526/Downloads/spellcheck/LRC-3.xlsx")
#t=t1
t=pd.concat([t4,t5,t6],ignore_index=True)
print(t.shape)
t['wgt']=0.0
t
t.columns=["Intent","Sentence","wgt"]


(5868, 2)


In [16]:
import pandas as pd
t=pd.read_excel("/Users/a656526/Downloads/spellcheck/LRC-Ver3 (1).xlsx")
t['len']=t['Sentence to Review'].str.split(" ").map(len)
t=t[t['len']>3]
t.drop(columns='len',inplace=True)
t.columns=["Sentence"]
t['Intent']=''
t['wgt']=0.0
t.head()
t=t[["Intent","Sentence","wgt"]].reset_index(drop=True)
t.head()

,Intent,Sentence,wgt
0,,I did not receive w-2 forms,0.0
1,,Cannot find my 1099-r on any of the fidelity site,0.0
2,,1099 has not arrived,0.0
3,,Why can not i get my 1099,0.0
4,,I did not get my w-2 form,0.0


In [24]:
#corpus=pd.concat([t1,t],ignore_index=True).sort_values(by='wgt',ascending=False).reset_index(drop=True)
corpus.shape

(8147, 4)

In [19]:
import pickle
import numpy as np
corpus['doc1']=np.arange(corpus.shape[0])
#f.drop(columns="doc",inplace=True)
corpus["doc"]="doc"+corpus["doc1"].astype(str)
corpus.drop(columns="doc1",inplace=True)
corpus.head()

,Intent,Sentence,wgt,doc
0,TAX_FORMS,When will I receive my tax form?,0.0311,doc0
1,I_WANT_TO_SET_UP_A_TRANSFER,I want to set up a transfer,0.0297,doc1
2,HOW_DO_I_MAKE_A_STOCK_OR_MUTUAL_FUND_TRADE,How do I make a trade?,0.0279,doc2
3,CONTRIBUTIONS,I have a question about my contributions,0.0267,doc3
4,I_WANT_TO_SET_UP_A_WITHDRAWAL,I want to set up a withdrawal,0.0230,doc4


In [25]:
out=dict(zip(corpus.doc,zip(corpus.Sentence,corpus.wgt)))
print(out)
with open("corpus.pickle","wb") as f:
    pickle.dump(out,f)
    f.close()

{'doc0': ('When will I receive my tax form?', 0.0311), 'doc1': ('I want to set up a transfer', 0.0297), 'doc2': ('How do I make a trade?', 0.0279), 'doc3': ('I have a question about my contributions', 0.0267), 'doc4': ('I want to set up a withdrawal', 0.023), 'doc5': ('I have questions about rollovers', 0.0219), 'doc6': ('I want to contact you', 0.0215), 'doc7': ('I have a question about Billpay', 0.0186), 'doc8': ('When will my funds be available?', 0.0183), 'doc9': ('I need information about loans', 0.018), 'doc10': ('I have a question about RMD', 0.017), 'doc11': ('I am having trouble logging in', 0.0149), 'doc12': ('I have a question about my beneficiaries', 0.0138), 'doc13': ('I have a question about a Fidelity debit or credit card', 0.0133), 'doc14': ('I need to update my personal information', 0.0114), 'doc15': ('How do I close my account?', 0.0113), 'doc16': ('I have questions about enrollment', 0.011), 'doc17': ('How do I connect a bank account to my Fidelity account?', 0.0105

In [13]:
list(set(res[-4]))

['Why is there a 125 fee to transfer money out?',
 "Why can't I transfer money from fidelity to bank account?",
 'Do I have to wait to transfer money into account.',
 "Can't transfer money to a bank acct.",
 'How to transfer money from tod account to joint account?',
 'Transfer money from an indian bank into fidelity account.',
 'I cannot transfer money from bank account through eft.',
 'There I want to transfer money between two 529 accounts.',
 "Why can't I transfer money?",
 'How do I transfer money to a new cd account?',
 'Transfer money to tsp.',
 'How do I transfer money from one fund to another?',
 'What is the best way to transfer money?',
 'Transfer money into se 401k.',
 'How do I transfer money to brother?',
 'I am unable to transfer money.',
 'Transfer money to me health savings account.',
 'Transfer money into childs college account.',
 'Can I transfer money to closed management account?',
 "I can't transfer money into managed roth ira.",
 "Why can't I transfer money into 

In [17]:
a

3402